In [1]:
import sys
print(sys.version)

3.8.5 (default, Jul 21 2020, 10:48:26) 
[Clang 11.0.3 (clang-1103.0.32.62)]


In [2]:
!ls /dev/cu*

/dev/cu.Bluedio-BluetoothSerial /dev/cu.RobinsiPhone6-Wirelessi
/dev/cu.Bluetooth-Incoming-Port


In [3]:
!otool -L /usr/local/lib/libedgetpu.1.dylib

/usr/local/lib/libedgetpu.1.dylib:
	/usr/local/lib/libedgetpu.1.dylib (compatibility version 0.0.0, current version 0.0.0)
	/usr/lib/libc++.1.dylib (compatibility version 1.0.0, current version 905.6.0)
	/usr/local/lib/libusb-1.0.0.dylib (compatibility version 4.0.0, current version 4.0.0)
	/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation (compatibility version 300.0.0, current version 1775.118.101)
	/usr/lib/libobjc.A.dylib (compatibility version 1.0.0, current version 228.0.0)
	/usr/lib/libSystem.B.dylib (compatibility version 1.0.0, current version 1292.100.5)
	/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation (compatibility version 150.0.0, current version 1775.118.101)


https://coral.ai/docs/edgetpu/tflite-python/#inferencing-example

To download the required files see https://github.com/google-coral/tflite/blob/master/python/examples/classification/install_requirements.sh

In [4]:
# !pip freeze

# crucially pycoral==2.0.0

In [5]:
import os
import pathlib
from pycoral.utils import edgetpu
from pycoral.utils import dataset
from pycoral.adapters import common
from pycoral.adapters import classify
from PIL import Image

Created https://github.com/google-coral/pycoral/issues/35

In [6]:
model_file = os.path.join('models', 'mobilenet_v2_1.0_224_inat_bird_quant_edgetpu.tflite')
# model_file = os.path.join('models', 'mobilenet_v2_1.0_224_inat_bird_quant.tflite')
print(os.path.isfile(model_file))

label_file = os.path.join('models', 'inat_bird_labels.txt')
print(os.path.isfile(label_file))

image_file = os.path.join('images', 'parrot.jpg')
print(os.path.isfile(image_file))

True
True
True


In [7]:
# Initialize the TF interpreter
interpreter = edgetpu.make_interpreter(model_file)
interpreter.allocate_tensors()

ValueError: Failed to load delegate from libedgetpu.1.dylib


In [ ]:
# Resize the image
size = common.input_size(interpreter)
image = Image.open(image_file).convert('RGB').resize(size, Image.ANTIALIAS)

# Run an inference
common.set_input(interpreter, image)
interpreter.invoke()
classes = classify.get_classes(interpreter, top_k=1)

# Print the result
labels = dataset.read_label_file(label_file)
for c in classes:
  print('%s: %.5f' % (labels.get(c.id, c.id), c.score))